In [1]:
using NeuralPDE, Lux, Plots, Random, Distributions, Optim , OptimizationOptimJL
using IntegralsCubature, FinancialToolbox
using ModelingToolkit: Interval
Random.seed!(0)




TaskLocalRNG()

## Model parameters

In [2]:

σ = 0.2*10 
α = 0.5
r = 0.1
K = 100 # strike is 100
T = 1 #one year 
Smin = 80
Smax = 120 # stock range make it 20% in-the-money and out-the-money
#scaling_factor = 0.05 #scaling factor that is good for boundary
scaling_factor = 0.04 #scaling factor that is good for interior
boundary_func(s) = max(s - K,0)*scaling_factor

boundary_func (generic function with 1 method)

## Call_pricer under CEV

In [3]:
κ(tau) = 2*r/(σ^(2*(1 - α))*(exp(2*r*(1-α)*tau)-1))
a(x,tau) = κ(tau)*x.^(2*(1 - α))*exp(2*r*(1 - α)*tau)
b(tau) = κ(tau)*K^(2*(1 - α))
c = 2 + (1/(1 - α))
F(x,tau) = Distributions.NoncentralChisq(c,a(x,tau))
G(tau) = Distributions.NoncentralChisq(c - 2,b(tau))
function CEV_call(x,tau)
    x*(1 - Distributions.cdf(F(x,tau), b(tau))) - K*exp(-r*tau)*Distributions.cdf(G(tau), a(x,tau))
end

CEV_call (generic function with 1 method)

## PDE under the CEV model

In [4]:
# 2D PDE in (t, s)
@parameters t s
@variables U(..)
Dt = Differential(t)
Ds = Differential(s)
Dss = Differential(s)^2

eq = Dt(U(t, s)) + r*s*Ds(U(t, s)) + 1/2*σ^2*s^(2*α)*Dss(U(t, s)) - r*U(t, s) ~ 0

# Boundary Conditions
boundary_conditions = [U(T, s) ~ boundary_func(s)]

# Problem Domain
domains = [t ∈ Interval(0, T), s ∈ Interval(Smin, Smax)]

2-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(t, 0..1)
 Symbolics.VarDomainPairing(s, 80..120)

In [5]:
dim = 2
chain = Lux.Chain(Dense(dim, 16, Lux.σ), Dense(16, 16, Lux.σ), Dense(16, 1))
ps = Lux.setup(Random.default_rng(), chain)[1]

# Transform PDESystem into OptimizationProblem using PINN methodology
strategy = QuadratureTraining()
discretization_algo = PhysicsInformedNN(chain, strategy, init_params=ps)
@named pde_system = PDESystem(eq, boundary_conditions, domains, [t, s], [U(t, s)])
optim_prob = discretize(pde_system, discretization_algo)

OptimizationProblem. In-place: true
u0: ComponentVector{Float32}(layer_1 = (weight = Float32[-0.5096706 0.42996323; -0.051686242 -0.19706658; … ; -0.23860326 5.093088f-6; 0.5504109 -0.03434386], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = Float32[0.15251705 0.39284882 … -0.022368534 0.20915501; -0.2943246 0.08905726 … 0.2225429 -0.1315318; … ; -0.42614084 -0.41936278 … -0.3245835 -0.14442606; 0.047960266 0.27465823 … 0.005657302 0.058415208], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = Float32[0.3952902 0.19821757 … -0.5427456 0.17304797], bias = Float32[0.0;;]))

In [6]:
using TickTock

In [7]:
tick()
callback = function (p,l)
    println("Current loss is: $l")
    return false
end


opt = BFGS()
result = Optimization.solve(optim_prob, opt, callback = callback, maxiters=2500, reltol=1e-6)
result.original
phi = discretization_algo.phi
tock()

[ Info:  started timer at: 2023-01-26T11:43:49.606


Current loss is: 2.4529366607924645
Current loss is: 1.5456420895437903
Current loss is: 1.4309264352228042
Current loss is: 1.424555653238035
Current loss is: 1.4186524478402989
Current loss is: 1.4179320008545473
Current loss is: 1.4096243338431738
Current loss is: 1.4048168073315466
Current loss is: 1.404553058301421
Current loss is: 1.4042095020110374
Current loss is: 1.4040148910134675
Current loss is: 1.4035690022271885
Current loss is: 1.4023936556409526
Current loss is: 1.401958722425166
Current loss is: 1.3990903394256422
Current loss is: 1.3952371310366503
Current loss is: 1.3893137145081007
Current loss is: 1.3660544323768111
Current loss is: 1.3376523192962346
Current loss is: 1.3322340231028715
Current loss is: 1.3301527030335942
Current loss is: 1.3270714403967856
Current loss is: 1.3166636861119552
Current loss is: 1.3165047344371992
Current loss is: 1.3096123302406768
Current loss is: 1.151231376447157
Current loss is: 0.5070209296040992
Current loss is: 0.5044097703809

[ Info:          86.19272125s: 1 minute, 26 seconds, 192 milliseconds


## 2-D Plots

In [ ]:
dx = 0.5
x_domain = Smin:dx:Smax
S_domain = collect(x_domain)

times = [0, 0.25, 0.5, 1]
tau = T .- times
plots = Array{Plots.Plot{Plots.GRBackend},1}()
for i in 1:(length(times))-1
    current_t = times[i]
    current_tau = tau[i]
    u_predict = [phi([current_t, current_x],result.u) for current_x in x_domain]
    u_predict = reduce(vcat, u_predict)/scaling_factor
    u_true = CEV_call.(S_domain, current_tau)
    push!(plots, plot(S_domain, [u_true, u_predict], label= ["CEV_True" "PINN"],legend=:topleft, title="Tau = $(current_tau)"))
    #push!(plots, plot(S_domain, V_predict, label=""))
end

u_predict = [phi([times[4], current_x],result.u) for current_x in x_domain]
u_predict = reduce(vcat, u_predict)/scaling_factor


u_true = collect(max.(S_domain .- K,0))
u_true = reduce(vcat, u_true)

push!(plots, plot(S_domain, [u_true, u_predict], label= ["CEV_True" "PINN"],legend=:topleft, title="Tau = 0"))



plot(plots[1], plots[2], plots[3], plots[4], layout = 4)

In [ ]:
u_predict1 = [phi([times[1], current_x],result.u) for current_x in x_domain]
u_predict1 = reduce(vcat, u_predict1)/scaling_factor
u_true1 = CEV_call.(S_domain, tau[1])

u_predict2 = [phi([times[2], current_x],result.u) for current_x in x_domain]
u_predict2 = reduce(vcat, u_predict2)/scaling_factor
u_true2 = CEV_call.(S_domain, tau[2])

u_predict3 = [phi([times[3], current_x],result.u) for current_x in x_domain]
u_predict3 = reduce(vcat, u_predict3)/scaling_factor
u_true3 = CEV_call.(S_domain, tau[3])

u_predict4 = [phi([times[4], current_x],result.u) for current_x in x_domain]
u_predict4 = reduce(vcat, u_predict4)/scaling_factor
u_true4 = collect(max.(S_domain .- K,0))
u_true4 = reduce(vcat, u_true)

In [ ]:
using PyPlot
using Makie # to enable save function 

#figure(1);
fig = figure(figsize=(10, 10), dpi=1200);
#subplots_adjust(hspace=0.5,wspace=0.9)
subplot(221) ;  PyPlot.plot(x_domain,u_predict1,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true1,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(222) ;  PyPlot.plot(x_domain,u_predict2,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true2,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.25",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(223) ;  PyPlot.plot(x_domain,u_predict3,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true3,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.5",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(224) ;  PyPlot.plot(x_domain,u_predict4,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true4,color = "magenta", label = "Analytical");
PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

fig
save("./Fig_7.png", fig)


In [ ]:
# first plot

 
#using LaTeXStrings

fig = Figure(resolution=(1400, 600), fontsize = 22)
ax1 = Axis3(fig[1, 1], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = "Call Price",
    title = "PINN")
ax2 = Axis3(fig[1, 2], perspectiveness=0.2, xlabel = "S", ylabel = "t", zlabel = " Call Price",
    title = "Analytical solution")

GLMakie.surface!(ax1, domain_s , domain_t', predict_matrix1 , transparency = true)
GLMakie.surface!(ax2, domain_s, domain_t', analytical_matrix1 , transparency = true)
save("./Fig_8.png", fig)
fig


## Implied Vol plots

In [ ]:


CEV_vol(x) = σ*x.^(α-1)
vols = CEV_vol(S_domain)

plot(x_domain,vols,color = "blue")

In [ ]:
using Roots

function CEV_vol(S0, price_d)
    f(x) = CEV_call(S0, 0) - price_d
    vol = find_zero(f , 0.7)
    return vol
end

In [ ]:
vols_vec = zeros(length(u_predict1),1)

for i in 1:length(u_predict1)
    vols_vec[i] = CEV_vol(S_domain[i], u_predict[i])
end

In [ ]:
plot(S_domain, vols_vec)